# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [3]:
# path to the model
dir_model = "../model/xyz_LSTM/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]
inferer??

Signature:      inferer(*args, **kwargs)
Type:           _WrapperFunction
String form:   
ConcreteFunction signature_wrapper_serving_default(*, inputs)
  Args:
    inputs: float32 Tensor, shape=(None, 20, 99)
  Returns:
    {'output_0': <1>}
      <1>: float32 Tensor, shape=(None, 2)
File:           c:\users\ahmad fadli\appdata\roaming\python\python311\site-packages\tensorflow\python\saved_model\load.py
Source:        
class _WrapperFunction(function.ConcreteFunction):
  """A class wraps a concrete function to handle different distributed contexts.

  The reason for wrapping a concrete function is because the _captured_inputs
  fields used for in-replica context and cross-replica context are different.
  When `load()` is called from within a tf.distribute.strategy scope, the
  captured inputs are distributed variables. When using these distributed
  variables during calling the function, we need different approaches when it is
  in-replica and when it is not in-replica. When it is in r

# 2. Setup OpenCV and MediaPipe to Predict

In [39]:
import time

In [48]:
dir_video_test = ('../../data/momentum/good4.mp4')
cap = cv2.VideoCapture(dir_video_test)
frame_rate = 10
prev = 0
current_stage = ''

i_momentum = 0
i_non_momentum = 0

sequences = []
#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        time_elapsed = time.time() - prev
        ret, frame = cap.read()
        if time_elapsed > 1./frame_rate:
            prev = time.time()

            # recolor feed
            image = cv2.resize(frame, (480, 640), interpolation= cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            # make detection
            results = pose.process(image)
            
            # recolor image back to BGR for rendering
            image.flags.writeable = False
            image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
    
            # try to predict
            try:
                # model xyz
                row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
                sequences.append(row)
                sequences  = sequences[-20:]
    
                if len(sequences) == 20:
                    momentum_class = np.argmax(inferer(inputs=(np.expand_dims(sequences, axis=0)))['output_0'].numpy())
                    momentum_prob = np.max(inferer(inputs=(np.expand_dims(sequences, axis=0)))['output_0'].numpy())
    
                # # model xyz_hand
                # # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
                # # X = pd.DataFrame([row])
                # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
                # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())
    
                # # model xy
                # # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
                # # X = pd.DataFrame([row])
                # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
                # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())
    
                # # model xy_hand
                # # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
                # # X = pd.DataFrame([row])
                # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
                # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())
    
                    print(momentum_class, momentum_prob)
    
                    if momentum_class == 0 and momentum_prob >= 0.9 :
                        current_stage = 'Nice!'
                        i_non_momentum+=1
                    elif momentum_class == 1 and momentum_prob >= 0.9 :
                        current_stage = 'Use momentum!'
                        i_momentum+=1
    
                    # get status box
                    cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
        
                    #Display class
                    cv2.putText(image, 'Class', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
                    #Display Prob
                    cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(round(momentum_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
            
            except Exception as e:
                print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

0 0.99967444
0 0.99943006
0 0.9986922
0 0.9981012
0 0.99894446
0 0.9994868
0 0.9997871
0 0.99997467
0 0.9999983
0 0.99999905
0 0.99999833
0 0.99999845
0 0.9999997
0 0.99999994
0 0.99999994
0 0.9999999
0 0.99999994
0 0.9999999
0 0.9999998
0 0.9999998
0 1.0
0 1.0
0 1.0
0 1.0
0 1.0
0 1.0


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [49]:
cap.release()
cv2.destroyAllWindows()